In [100]:
from gurobipy import *
import numpy as np
import math as mt

In [101]:
n1=3
p1=6

In [102]:
def random_matrice_utilite(n, p):
        u=np.zeros(6)
        u1= np.array([325,225,210,115,75,50])
        for i in range(n):
                u=np.vstack((u1,u))

        u=np.delete(u, (n), axis=0)
        return u

In [103]:
def generate_w_prime(n):
        w=[i for i in range(n,0,-1)]
        w1=[]
        for i in range(n):
                if i==n-1 :
                        w1.append(w[i])
                else :
                        w1.append(w[i]-w[i+1])
        return w1

In [104]:
def matrice_contrainte(n, p, u):
        # Matrice des contraintes
        # 1) generer la matrice des contraintes des x
        L=np.array([])
        L2=np.zeros((6,1))
        b=np.zeros(n*p)
        nbvar = n * (n + p + 1)
        u_prim=-1*u

        for i in range (n):
                for j in range(p):
                        L=np.append(L,[u_prim[i,j]])
                for k in range(nbvar-(n*p)):
                        L=np.append(L,0)

                b=np.vstack((L,b))  
                L=np.array([])
        b=np.delete(b, (n), axis=0)

        for i in range (n):
                for k in range(p):
                        L=np.append(L,0)
                for j in range(p):
                        L=np.append(L,[u_prim[i,j]])
                for k in range(p):
                        L=np.append(L,0)

                b=np.vstack((b,L)) 
                L=np.array([])

        for i in range (n):
                for k in range(nbvar-(n*p)):
                        L=np.append(L,0)
                for j in range(p):
                        L=np.append(L,[u_prim[i,j]])
        
                b=np.vstack((b,L))  
                L=np.array([])


        for i in range(n):
                L1=np.identity(p)
                L2=np.concatenate((L1,L2),axis=1)
        L2=np.delete(L2,[-1],1)
        b=np.vstack((b,L2))



        # 2) generer la matrice des contraintes des bik
        L=np.identity(n*n)
        L=-1*L
        for i in range (p):
                L1=np.zeros(n*n)
                L =np.vstack((L,L1))

        # 2) concatener la matrice des contraintes des bik et des xj dans la matrice b

        b=np.concatenate((L,b),axis=1)

        # 3) generer la matrice des contraintes des rk
        L2=np.zeros((1,n))
        for i in range(n):
                L1=np.identity(n)
                L2=np.vstack((L1,L2))
        L2=np.delete(L2,[-1],0)
        L1=np.zeros((p,n))
        L2=np.vstack((L2,L1))
        
        # 2) concatener la matrice des contraintes rk et b
        b=np.concatenate((L2,b),axis=1)
        return b

In [105]:
def second_membre(p):
        S= [0 for i in range (9)]
        for i in range(p):
                S.append(1)
        return S

In [106]:
def coef_func_objectif(n, p, w1):
        # Coefficients de la fonction objectif
        # generer les Coefficients des r
        a=[]
        for i in range(1,n+1):
                a.append(i*w1[i-1])
        # generer les Coefficients des bik
        for i in range(n):
                for j in range(n):
                        a.append(-1*w1[j])
        # generer les Coefficients des x
        for i in range (p*n):
                a.append(0)
        return a

In [107]:
# Creer les models de la fonction objectif
m = Model("Experimentation partie 2") 

In [108]:
def declaration_vars(n, p, m):
        # declaration des variables de decisions
        x = []
        for i in range(n):
                x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="r%d" % (i+1)))
        for i in range (n):
                for j in range(n):
                        x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="b%d%d" % ((i+1,(j+1)))))
        for i in range (n):
                for j in range(p):
                        x.append(m.addVar(vtype=GRB.BINARY, lb=0, name="x%d%d" % ((i+1,(j+1)))))

        # maj du modele pour integrer les nouvelles variables
        m.update()
        return x

In [109]:
def definition_objectif(n, p, a, x, m):
        # definition de l'objectif
        obj = LinExpr();
        obj =0
        for j in range(n * (n + p +1)):
                obj += a[j] * x[j]

        m.setObjective(obj,GRB.MAXIMIZE)
        return obj

In [110]:
def definition_contraintes(n, p, m, b, x, S):
        # Definition des contraintes
        lignes = range(n*n+p)
        colonnes = range(n*(n+p+1))

        for i in lignes:
                m.addConstr(quicksum(b[i][j]*x[j] for j in colonnes) <= S[i], "Contrainte%d" % i)

In [111]:
def display_vars(m):
        print("")                
        print('Solution optimale:')
        mvars = m.getVars()
        names = m.getAttr('VarName', mvars)
        values = m.getAttr('X', mvars)
        result = dict(zip(names, values))
        for key in result:
                print(key,'=',result[key])
        print("")
        print('Valeur de la fonction objectif :', m.objVal)


In [113]:
def experience(n1, p1):
        u = random_matrice_utilite(n1, p1)
        w1 = generate_w_prime(n1)
        b = matrice_contrainte(n1, p1, u)
        S = second_membre(p1)
        a = coef_func_objectif(n1, p1, w1)
        # Creer les models de la fonction objectif
        m = Model("Experimentation partie 2") 
        x = declaration_vars(n1, p1, m)
        obj = definition_objectif(n1, p1, a, x, m)
        definition_contraintes(n1, p1, m, b, x, S)
        # Resolution
        m.optimize()
        display_vars(m)

experience(3, 6)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15 rows, 30 columns and 90 nonzeros
Model fingerprint: 0xb0cb6945
Variable types: 12 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 15 rows, 30 columns, 90 nonzeros
Variable types: 12 continuous, 18 integer (18 binary)

Root relaxation: objective 2.000000e+03, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2000.00000    0    4   -0.00000 2000.00000      -     - 